In [ ]:
"""
File: Bi-LSTM.ipynb
Code to train and evaluate a bi-directional LSTM model on MIMIC-IV FHIR dataset.
"""


def Project():
    """
    __Objectives__
    0. Import data and tokenizer
    1. Train the tokenizer on all sequences of the dataset
    2. Tokenize different sequences and join them together
    3. Prepare actual labels for one, six, twelve month death after discharge
    4. Define the model architecture for bidrectional LSTM
    5. Train Bi-LSTM model and evaluate on test dataset
    6. Compare performance across new tasks to XGBoost
    >>> All objectives successful
    """
    return ProjectObjectives.__doc__

In [ ]:
import os
import sys


ROOT = "/fs01/home/afallah/odyssey/odyssey"
os.chdir(ROOT)

from typing import Any, Dict, Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from sklearn.metrics import (
    auc,
    average_precision_score,
    balanced_accuracy_score,
    f1_score,
    precision_recall_curve,
    precision_score,
    recall_score,
    roc_auc_score,
    roc_curve,
)
from torch import nn, optim
from torch.nn.functional import sigmoid
from torch.nn.utils.rnn import pack_padded_sequence
from torch.optim.lr_scheduler import ExponentialLR
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

from odyssey.data.dataset import FinetuneDataset
from odyssey.data.tokenizer import ConceptTokenizer
from odyssey.models.cehr_big_bird.embeddings import Embeddings


DATA_ROOT = f"{ROOT}/data/slurm_data/512/one_month"
DATA_PATH = f"{DATA_ROOT}/pretrain.parquet"
FINE_TUNE_PATH = f"{DATA_ROOT}/fine_tune.parquet"
TEST_DATA_PATH = f"{DATA_ROOT}/fine_test.parquet"

%matplotlib inline

In [ ]:
# save parameters and configurations


class config:
    """A simple class to store all configurations"""

    seed = 23
    data_dir = DATA_ROOT
    test_size = 0.2
    batch_size = 64
    num_workers = 3
    vocab_size = None
    embedding_size = 768
    time_embeddings_size = 32
    type_vocab_size = 8
    max_len = 512
    padding_idx = None
    device = torch.device("cuda")


def seed_all(seed: int) -> None:
    """Seed all parts of the training process"""
    torch.manual_seed(seed)
    np.random.seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


seed_all(config.seed)
print(f"Cuda: {torch.cuda.get_device_name(torch.cuda.current_device())}")

In [ ]:
# Load data
pretrain_data = pd.read_parquet(DATA_PATH)
pretrain_data = pretrain_data[pretrain_data["event_tokens_512"].notnull()]

finetune_data = pd.read_parquet(FINE_TUNE_PATH)
finetune_data = finetune_data[finetune_data["event_tokens_512"].notnull()]

test_data = pd.read_parquet(TEST_DATA_PATH)
test_data = test_data[test_data["event_tokens_512"].notnull()]
test_length = len(test_data)

train_data = pd.concat((pretrain_data, finetune_data))
train_data.reset_index(inplace=True)
train_data.drop_duplicates(subset="index", keep="first", inplace=True).set_index(
    "index",
)

del pretrain_data, finetune_data

train_data

In [ ]:
# Define custom labels if dataset does not already have labels
# data["label"] = (
#     (data["death_after_end"] >= 0) & (data["death_after_end"] < 365)
# ).astype(int)

In [ ]:
# Fit tokenizer on .json vocab files
tokenizer = ConceptTokenizer(data_dir=config.data_dir)
tokenizer.fit_on_vocab()
config.vocab_size = tokenizer.get_vocab_size()
config.padding_idx = tokenizer.get_pad_token_id()
tokenizer

In [ ]:
# Define dataset with token lengths


class DatasetWithTokenLength(Dataset):
    def __init__(self, tokenized_data: FinetuneDataset, length_data: np.ndarray):
        super(Dataset, self).__init__()

        self.tokenized_data = tokenized_data
        self.length_data = length_data

        assert len(tokenized_data) == len(
            length_data,
        ), "Datasets have different lengths"

        self.sorted_indices = sorted(
            range(len(length_data)),
            key=lambda x: length_data[x],
            reverse=True,
        )

    def __len__(self) -> int:
        return len(self.tokenized_data)

    def __getitem__(self, index: int) -> Tuple[Any, int]:
        index = self.sorted_indices[index]
        return self.tokenized_data[index], min(config.max_len, self.length_data[index])


# Get training and test datasets

# Split data if needed
# train_data, test_data = train_test_split(
#     data, test_size=config.test_size, random_state=config.seed, stratify=data["label"]
# )

# Get train and test datasets and dataloaders
train_dataset = FinetuneDataset(
    data=train_data,
    tokenizer=tokenizer,
    max_len=config.max_len,
)

test_dataset = FinetuneDataset(
    data=test_data,
    tokenizer=tokenizer,
    max_len=config.max_len,
)

train_dataset_with_lengths = DatasetWithTokenLength(
    train_dataset,
    train_data["token_length"].values,
)
test_dataset_with_lengths = DatasetWithTokenLength(
    test_dataset,
    test_data["token_length"].values,
)

train_loader = DataLoader(
    train_dataset_with_lengths,
    batch_size=config.batch_size,
    num_workers=config.num_workers,
    shuffle=False,
    pin_memory=True,
)

test_loader = DataLoader(
    test_dataset_with_lengths,
    batch_size=config.batch_size,
    num_workers=config.num_workers,
    shuffle=False,
    pin_memory=True,
)

print("Data is ready to go!\n")

In [ ]:
# Define model architecture
class BiLSTMModel(nn.Module):
    def __init__(
        self,
        embedding_dim: int,
        hidden_size: int,
        num_layers: int,
        output_size: int,
        dropout_rate: float,
    ):
        super(BiLSTMModel, self).__init__()

        self.embeddings = Embeddings(
            vocab_size=config.vocab_size,
            embedding_size=config.embedding_size,
            time_embedding_size=config.time_embeddings_size,
            type_vocab_size=config.type_vocab_size,
            max_len=config.max_len,
            padding_idx=config.padding_idx,
        )

        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout_rate,
        )

        self.batch_norm = nn.BatchNorm1d(hidden_size * 2)
        self.dropout = nn.Dropout(dropout_rate)
        self.linear = nn.Linear(hidden_size * 2, output_size)

    def forward(self, inputs: Tuple[Any], lengths: torch.Tensor) -> torch.Tensor:
        """Forward pass of Bi-LSTM model"""
        embed = self.embeddings(*inputs)
        packed_embed = pack_padded_sequence(embed, lengths.cpu(), batch_first=True)

        lstm_out, (hidden_state, cell_state) = self.lstm(packed_embed)
        output = torch.cat((hidden_state[-2, :, :], hidden_state[-1, :, :]), dim=1)

        output = self.dropout(self.batch_norm(output))
        output = self.linear(output)
        return output

    @staticmethod
    def get_inputs_labels(
        sequences: Dict[str, torch.Tensor],
    ) -> Tuple[Any, torch.Tensor]:
        """Create inputs tuples from a dictionary of sequences"""
        labels = sequences["labels"].view(-1, 1).to(config.device)
        inputs = (
            sequences["concept_ids"].to(config.device),
            sequences["type_ids"].to(config.device),
            sequences["time_stamps"].to(config.device),
            sequences["ages"].to(config.device),
            sequences["visit_orders"].to(config.device),
            sequences["visit_segments"].to(config.device),
        )

        return inputs, labels.float()

    @staticmethod
    def get_balanced_accuracy(outputs: torch.Tensor, labels: torch.Tensor) -> float:
        """Return the balanced accuracy metric by comparing outputs to labels"""
        predictions = torch.round(sigmoid(outputs))
        predictions = predictions.detach().cpu().numpy()
        labels = labels.detach().cpu().numpy()

        return balanced_accuracy_score(labels, predictions)

In [ ]:
# Set hyperparameters for Bi-LSTM model adn training loop
input_size = config.embedding_size  # embedding_dim
hidden_size = config.embedding_size // 2  # output hidden size
num_layers = 5  # number of LSTM layers
output_size = 1  # Binary classification, so output size is 1
dropout_rate = 0.2  # Dropout rate for regularization

# Set training hyperparameters
epochs = 6
learning_rate = 0.001

In [ ]:
# Training Loop
model = BiLSTMModel(input_size, hidden_size, num_layers, output_size, dropout_rate).to(
    config.device,
)
class_weights = torch.tensor([6]).to(config.device)  # Determined with experiment
loss_fcn = nn.BCEWithLogitsLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = ExponentialLR(optimizer, gamma=0.75, verbose=True)

for epoch in range(epochs):
    train_total_loss = 0
    train_accuracy = 0
    test_accuracy = 0

    model.train()
    for batch_no, (sequences, lengths) in tqdm(
        enumerate(train_loader),
        file=sys.stdout,
        total=len(train_loader),
        desc=f"Epoch {epoch + 1}/{epochs}",
        unit=" batch",
    ):
        inputs, labels = model.get_inputs_labels(sequences)
        outputs = model(inputs, lengths)
        loss = loss_fcn(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_total_loss += loss.item()

    # Run an evaluation loop if needed
    # model.eval()
    # with torch.no_grad():
    #     for batch_no, (sequences, lengths) in tqdm(
    #         enumerate(train_loader),
    #         file=sys.stdout,
    #         total=len(train_loader),
    #         desc=f"Train Evaluation {epoch + 1}/{epochs}",
    #         unit=" batch",
    #     ):
    #         inputs, labels = model.get_inputs_labels(sequences)
    #         outputs = model(inputs, lengths)
    #         train_accuracy += model.get_balanced_accuracy(outputs, labels)
    #
    #     for batch_no, (sequences, lengths) in tqdm(
    #         enumerate(test_loader),
    #         file=sys.stdout,
    #         total=len(test_loader),
    #         desc=f"Test Evaluation {epoch + 1}/{epochs}",
    #         unit=" batch",
    #     ):
    #         inputs, labels = model.get_inputs_labels(sequences)
    #         outputs = model(inputs, lengths)
    #         test_accuracy += model.get_balanced_accuracy(outputs, labels)

    print(
        f"\nEpoch {epoch + 1}/{epochs}"
        f"  |  Average Train Loss: {train_total_loss / len(train_loader):.5f}"
        f"  |  Train Accuracy: {train_accuracy / len(train_loader):.5f}",
        # f"  |  Test Accuracy: {test_accuracy / len(test_loader):.5f}\n\n"
        "\n\n",
    )
    scheduler.step()

In [ ]:
# Save the model if needed
# torch.save(model, 'LSTM_V2.pt')

In [ ]:
# Load saved model from disk
state_dict = torch.load(
    "/fs01/home/afallah/odyssey/slurm/LSTM_V4_Weighted1.pt",
).state_dict()

model = BiLSTMModel(input_size, hidden_size, num_layers, output_size, dropout_rate).to(
    config.device,
)

model.load_state_dict(state_dict)

In [ ]:
# Assess model performance and get predictions for train and test set
y_train_pred = np.array([])
y_train_labels = np.array([])
y_test_pred = np.array([])
y_test_labels = np.array([])
y_test_prob = np.array([])


model.eval()
with torch.no_grad():
    # Get predictions on training data
    # for batch_no, (sequences, lengths) in tqdm(
    #     enumerate(train_loader),
    #     file=sys.stdout,
    #     total=len(train_loader),
    #     desc="Train Evaluation",
    #     unit=" batch",
    # ):
    #     inputs, labels = model.get_inputs_labels(sequences)
    #     outputs = model(inputs, lengths)
    #
    #     predictions = torch.round(sigmoid(outputs))
    #     predictions = predictions.detach().cpu().numpy()
    #     labels = labels.detach().cpu().numpy()
    #
    #     y_train_pred = np.append(y_train_pred, predictions)
    #     y_train_labels = np.append(y_train_labels, labels)

    # Get predictions on test data
    for batch_no, (sequences, lengths) in tqdm(
        enumerate(test_loader),
        file=sys.stdout,
        total=len(test_loader),
        desc="Test Evaluation",
        unit=" batch",
    ):
        inputs, labels = model.get_inputs_labels(sequences)
        outputs = model(inputs, lengths).detach().cpu()

        predictions = torch.round(sigmoid(outputs))
        predictions = predictions.detach().cpu().numpy()
        labels = labels.detach().cpu().numpy()

        y_test_pred = np.append(y_test_pred, predictions)
        y_test_labels = np.append(y_test_labels, labels)
        y_test_prob = np.append(y_test_prob, sigmoid(outputs))

# Save test and training data prediction together
# all_data_pred = np.append(y_train_pred, y_test_pred)
# all_data_labels = np.append(y_train_labels, y_test_labels)

In [ ]:
# Save model predictions, labels, and probabilities to disk
np.save(f"{ROOT}/lstm_y_test_pred_one_month.npy", y_test_pred)
np.save(f"{ROOT}/lstm_y_test_pred_one_month_labels.npy", y_test_labels)
np.save(f"{ROOT}/lstm_y_test_pred_one_month_prob.npy", y_test_prob)

In [ ]:
### ASSESS MODEL PERFORMANCE ###

# Balanced Accuracy
y_train_accuracy = balanced_accuracy_score(y_train_labels, y_train_pred)
y_test_accuracy = balanced_accuracy_score(y_test_labels, y_test_pred)
all_data_accuracy = balanced_accuracy_score(all_data_labels, all_data_pred)

# F1 Score
y_train_f1 = f1_score(y_train_labels, y_train_pred)
y_test_f1 = f1_score(y_test_labels, y_test_pred)
all_data_f1 = f1_score(all_data_labels, all_data_pred)

# Precision
y_train_precision = precision_score(y_train_labels, y_train_pred)
y_test_precision = precision_score(y_test_labels, y_test_pred)
all_data_precision = precision_score(all_data_labels, all_data_pred)

# Recall
y_train_recall = recall_score(y_train_labels, y_train_pred)
y_test_recall = recall_score(y_test_labels, y_test_pred)
all_data_recall = recall_score(all_data_labels, all_data_pred)

# AUROC
y_train_auroc = roc_auc_score(y_train_labels, y_train_pred)
y_test_auroc = roc_auc_score(y_test_labels, y_test_pred)
all_data_auroc = roc_auc_score(all_data_labels, all_data_pred)

# AUC-PR (Area Under the Precision-Recall Curve)
y_train_p, y_train_r, _ = precision_recall_curve(y_train_labels, y_train_pred)
y_test_p, y_test_r, _ = precision_recall_curve(y_test_labels, y_test_pred)
all_data_p, all_data_r, _ = precision_recall_curve(all_data_labels, all_data_pred)

y_train_auc_pr = auc(y_train_r, y_train_p)
y_test_auc_pr = auc(y_test_r, y_test_p)
all_data_auc_pr = auc(all_data_r, all_data_p)

# Average Precision Score (APS)
y_train_aps = average_precision_score(y_train_labels, y_train_pred)
y_test_aps = average_precision_score(y_test_labels, y_test_pred)
all_data_aps = average_precision_score(all_data_labels, all_data_pred)

# Print Metrics
print(
    f"Balanced Accuracy\nTrain: {y_train_accuracy:.5f}  |  Test: {y_test_accuracy:.5f}  |  All Data: {all_data_accuracy:.5f}\n",
)
print(
    f"F1 Score\nTrain: {y_train_f1:.5f}  |  Test: {y_test_f1:.5f}  |  All Data: {all_data_f1:.5f}\n",
)
print(
    f"Precision\nTrain: {y_train_precision:.5f}  |  Test: {y_test_precision:.5f}  |  All Data: {all_data_precision:.5f}\n",
)
print(
    f"Recall\nTrain: {y_train_recall:.5f}  |  Test: {y_test_recall:.5f}  |  All Data: {all_data_recall:.5f}\n",
)
print(
    f"AUROC\nTrain: {y_train_auroc:.5f}  |  Test: {y_test_auroc:.5f}  |  All Data: {all_data_auroc:.5f}\n",
)
print(
    f"AUC-PR\nTrain: {y_train_auc_pr:.5f}  |  Test: {y_test_auc_pr:.5f}  |  All Data: {all_data_auc_pr:.5f}\n",
)
print(
    f"Average Precision Score\nTrain: {y_train_aps:.5f}  |  Test: {y_test_aps:.5f}  |  All Data: {all_data_aps:.5f}\n",
)

# Plot ROC Curve
fpr_train, tpr_train, _ = roc_curve(y_train_labels, y_train_pred)
fpr_test, tpr_test, _ = roc_curve(y_test_labels, y_test_pred)
fpr_all_data, tpr_all_data, _ = roc_curve(all_data_labels, all_data_pred)

# Plot Information
plt.figure(figsize=(10, 7))
plt.plot(fpr_train, tpr_train, label=f"Train AUROC={y_train_auroc:.2f}")
plt.plot(fpr_test, tpr_test, label=f"Test AUROC={y_test_auroc:.2f}")
plt.plot(fpr_all_data, tpr_all_data, label=f"All Data AUROC={all_data_auroc:.2f}")
plt.plot([0, 1], [0, 1], linestyle="--", color="gray", label="Random")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic (ROC) Curve")
plt.legend()
plt.show()